In [1]:
# import torch
from transformers import BertTokenizer #, BertForSequenceClassification, DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.model_selection import train_test_split
# from torch.utils.data import Dataset, DataLoader
# import itertools
from tensorflow.nn import softmax
import re
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
# import ast
# from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import balanced_accuracy_score,f1_score,accuracy_score #pip install -U scikit-learn
# import matplotlib.pyplot as plt
# import seaborn as sns
np.random.seed(42)



/Users/bikramkhanal/miniconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the pretrained model
model = TFBertForSequenceClassification.from_pretrained('model.h5')

2024-01-31 17:54:28.638674: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-31 17:54:28.638702: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-01-31 17:54:28.638709: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-01-31 17:54:28.638981: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-31 17:54:28.639410: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some layers from the model checkpoint at model.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertFo

In [14]:
# load the test data
data = []
with open("test.csv", 'r') as csvfile:
    for line in csvfile:
        if len(line) > 5:
            a, b = line.split('",', 1)
            a, b = a[2:], b[2:-2]
            bs = re.findall(r"'(.*?)'", b, re.DOTALL)
            bs = ", ".join(bs)
            data.append([a,bs])
# data = data[:10000]
len(data)

100

In [17]:
# separate the data into the text and the labels
texts = [i[0] for i in data]
labels = [i[1] for i in data]
real_labels = [i[1] for i in data]
#tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_ids = [tokenizer.encode(c,add_special_tokens=True) for c in texts]
#pad
input_ids = pad_sequences(input_ids, maxlen=256, dtype="long", value=0, truncating="post", padding="post")
# Initialze the label encoder
le = LabelEncoder()
# Fit the label encoder
le.fit(labels)
# Transform the labels
labels = le.transform(labels)
# One hot encode the labels
labels = to_categorical(labels)

In [ ]:
# make predictions
preds = model.predict(input_ids)
logits = preds.logits
# get the predicted labels
preds = np.argmax(logits, axis=1)
# get the accuracy
preds = le.inverse_transform(preds)
df['preds'] = preds
